<a href="https://colab.research.google.com/github/saifulrijal-ds/llm-zoomcamp-2024/blob/main/02-open-source/colab/flan-t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers accelerate bitsandbytes

In [2]:
!nvidia-smi

Sat Jul  6 19:17:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-06 18:34:48--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-06 18:34:49 (50.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import requests
import minsearch

In [4]:
from ast import keyword
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
  course_name = course['course']

  for doc in course['documents']:
    doc['course'] = course_name
    documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search(query):
  boost = {'question':3.0, 'section': 0.5}

  results = index.search(
      query=query,
      filter_dict={'course': 'data-engineering-zoomcamp'},
      boost_dict=boost,
      num_results=5
  )

  return results

In [6]:
def build_prompt(query, search_result):
  prompt_template = """
  You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

  context = ""

  for doc in search_result:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

  prompt = prompt_template.format(question=query, context=context).strip()
  return prompt

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [10]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-xl",
    device_map='auto',
    torch_dtype=torch.bfloat16
    )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
!nvidia-smi

Sat Jul  6 18:57:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0              26W /  70W |  10977MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
!nvidia-smi

Sat Jul  6 19:22:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0              26W /  70W |   6117MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
input_text = "translate English to Indonesian: How old are you?"
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to("cuda")
input_ids

tensor([[13959,  1566,    12,  9995,    29,    10,   571,   625,    33,    25,
            58,     1]], device='cuda:0')

In [13]:
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Jadi jumlah tu?</s>


In [14]:
!nvidia-smi

Sat Jul  6 19:22:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              26W /  70W |   6171MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
def llm(prompt, generate_params=None):
  if generate_params is None:
    generate_params = {}

  input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to("cuda")
  outputs = model.generate(
      input_ids,
      max_length=generate_params.get('max_length', 100),
      num_beams=generate_params.get('num_beams', 5),
      do_sample=generate_params.get('do_sample', True),
      temperature=generate_params.get('temperature', 1.0),
      top_k=generate_params.get('top_k', 50),
      top_p=generate_params.get('top_p', 0.95)
  )
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [16]:
def rag(query):
  search_result = search(query)
  prompt = build_prompt(query, search_result)
  answer = llm(prompt)
  return answer

In [17]:
llm("I just discovered the course. Can I still join in?")

'Yes, you are welcome to join us.'

In [18]:
rag("I just discovered the course. Can I still join in?")

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."